In [1]:
# Removed Multithread Processing from Read Directory. Need to have displine and Principles related to 
# Creation of Functions, Create Individual functions which do components, opposed to long ones. 

In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

In [2]:
from shared_folder import ReadDirectory
d_py_function =  '/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/'
ReadDirectory(d_py_function)

# Import a .TXT or .PY File
location = '/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/shared_folder.py'

from shared_folder import TextFileImport
file = TextFileImport(location)

from shared_folder import ParseDDotPYFile
function_list, function_parameters = ParseDDotPYFile(file)

## Generate a Summary File for a Folder.
from shared_folder import ParseDDotPYFolder

function_list, function_parameters = ParseDDotPYFolder()

In [4]:
function_list

,Function,Purpose,Parameters,Returns,date_created,date_last_modified,classification,sub_classification,usage,Folder
0,convert_dict_to_parameters,Convert a .py file containing Lists into a Dat...,[module],Dataframe,4-Dec-25,4-Dec-25,TBD,TBD,import d_dictionaries\ntemp = convertlisttopar...,dict_processing.py
1,dict_to_dataframe,Function to Simplify the creation of a Diction...,"[dict_, key_name, value_name]",Dataframe,4-Dec-25,4-Dec-25,TBD,TBD,temp_df = dict_to_dataframe(dict),dict_processing.py
0,ReadDirectory,Function which reads reads a directory and ret...,"[location, file_type, match_str]",Dataframe containing a listing of selected files.,3-Dec-25,3-Dec-25,TBD,TBD,d_py_function = '/Users/derekdewald/Documents...,shared_folder.py
1,TextFileImport,Function Used to Import .txt or .py File into ...,"[file_name, encoding]",Str,3-Dec-25,3-Dec-25,TBD,TBD,location = '/Users/derekdewald/Documents/Pytho...,shared_folder.py
2,ParseDDotPYFile,Function which reads a Python file (as text) a...,[file_text],function_list (DataFrame): One row per functio...,4-Dec-25,4-Dec-25,TBD,TBD,"function_list, function_parameters = ParseDDot...",shared_folder.py
3,ParseDDotPYFolder,Function which Allows for the Quick Review of ...,[location],DataFrame,4-Dec-25,4-Dec-25,TBD,TBD,"function_list, function_parameters = ParseDDot...",shared_folder.py
4,create_py_table_dict,Function which Generates a Dataframe represent...,[base_location],DataFrame,4-Dec-25,4-Dec-25,TBD,TBD,python_function_dict_df = create_py_table_dict(),shared_folder.py
0,convert_list_to_parameters,Convert a .py file containing Lists into a Dat...,[module],Dataframe,4-Dec-25,4-Dec-25,TBD,TBD,import d_lists\ntemp = convertlisttoparameters...,list_processing.py
1,list_to_dataframe,Function to Simplify the creation of a Diction...,"[list_, column_name_list]",Object Type,4-Dec-25,4-Dec-25,TBD,TBD,temp_df = list_to_dataframe(dict),list_processing.py
0,convert_str_to_parameters,Convert a .py file containing Strings into a D...,[module],Dataframe,4-Dec-25,4-Dec-25,TBD,TBD,import d_strings`aaax\ntemp = convertlisttopar...,string_processing.py


In [6]:
function_parameters

,Function,Parameters,Type,Definition,Folder
0,convert_dict_to_parameters,module,module,Py file containing Lists.,dict_processing.py
1,dict_to_dataframe,dict_,,,dict_processing.py
2,dict_to_dataframe,key_name,str,Name of Column which will include values from ...,dict_processing.py
3,dict_to_dataframe,value_name,str,Name of Column which will include values from ...,dict_processing.py
0,ReadDirectory,location,str,The path to the directory. Defaults to the cur...,shared_folder.py
1,ReadDirectory,file_type,str,"The file extension or type to filter by (e.g.,...",shared_folder.py
2,ReadDirectory,match_str,str,,shared_folder.py
3,TextFileImport,file_name,,,shared_folder.py
4,TextFileImport,encoding,,,shared_folder.py
5,ParseDDotPYFile,file_text,str,Full text of a .py file.,shared_folder.py


In [11]:
from shared_folder import create_py_table_dict

create_py_table_dict()

,File Name,Source,Function Name,Definition,Type
0,connections.py,PY File,connections,NaN,NaN
1,data_d_dicts.py,PY File,data_d_dicts,Repository of Dictionaries which have been sav...,Definition
2,data_d_lists.py,PY File,data_d_lists,Repository of Lists which have been saved for ...,Definition
3,data_d_strings.py,PY File,data_d_strings,Repository of Strings which have been saved fo...,Definition
4,df_processing.py,PY File,df_processing,NaN,NaN
5,dict_processing.py,PY File,dict_processing,"Functions related to Manipulating, Transformin...",Definition
6,list_processing.py,PY File,list_processing,"Functions related to Manipulating, Transformin...",Definition
7,shared_folder.py,PY File,shared_folder,"Functions related to Management, Maintenance a...",Definition
8,sql_.py,PY File,sql_,Functions related to Processing of SQL,Definition
9,string_processing.py,PY File,string_processing,"Functions related to Manipulating, Transformin...",Definition


In [ ]:
Machine Learning
- Linear Regression
-- Definition
-- Closed Form
-- Standardization
-- Other Stuff

### Process 

In [ ]:
import pandas as pd
import webbrowser
import datetime
import requests
import os

def DownloadFilesFromGit(user='derek-dewald',
                        repo='Python_Tools',
                        folder='d_py_functions',
                        output_folder=""):
    '''
    Function to Download Files from Github to a dedicated folder. Specifically used when i DO NOT want to formally link to Github.
    
    Parameters:
        User:
        Repo:
        folder:
        output_folder:
        
    Returns:
        Saves files to Output Folder.
    

    '''
    
    if len(output_folder) == 0:
        output_folder = os.getcwd()
    
    api_url = f"https://api.github.com/repos/{user}/{repo}/contents/{folder}"
    response = requests.get(api_url)

    if response.status_code == 200:
        files = response.json()
        py_files = [file for file in files if file['name'].endswith('.py')]

        for file in py_files:
            file_url = file['download_url']
            file_name = file['name']
            file_response = requests.get(file_url)

            if file_response.status_code == 200:
                with open(os.path.join(output_folder, file_name), "w", encoding="utf-8") as f:
                    f.write(file_response.text)
                    
        return True
    else:
        return False

def ParamterMapping(Definition=""):
    
    '''
    Function to Google Mapping Sheet, which is used to store Mappings, Links, etc.
    For both simplicity and Organization
    
    Args:
        Definition (Str): Key word used to Access individual elements
        
    Returns:
        Dataframe, unless Definition is defined, in which case it might be Str.
    
    '''

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSwDznLz-GKgWFT1uN0XZYm3bsos899I9MS-pSvEoDC-Cjqo9CWeEuSdjitxjqzF3O39LmjJB0_Fg-B/pub?output=csv')
    
    # If user has not included a definition, the return entire DF
    if len(Definition)==0:
        return df
    else:
        try:
            df1 = df[df['Definition']==Definition]
            if len(df1)==1:
                if df1['TYPE'].item()=='csv':
                    return pd.read_csv(df1['VALUE'].item())
                else:
                    return df1['VALUE'].item()
        except:
            return df[df['Definition']==Definition] 


def BackUpGoogleSheets(location='/Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/'):
    '''
    Function to Create a Backup of Information Stored in Google Sheets.
    
    Parameters:
        None
        
    Returns:
        CSV Files 
    
    '''
    
    df = ParamterMapping()
    
    for row in range(len(df)):
        try:
            file_name = df['Definition'][row]
            file_location = df['CSV'][row]
            month = datetime.datetime.now().strftime('%b-%y')
            
            temp_df = pd.read_csv(file_location)
            temp_df.to_csv(f'{location}{file_name}_{month}.csv',index=False)
            print(f'Back Up Saved, {location}{file_name}')
        except:
            print(f'Counld Not Print Record {row}')



def GoogleProcessSheetLinks():
     
    '''
    Function to Google Mapping Sheet, Navigate to Specific Sites.
    Provides Options, Enable Selection based on inputs.
    
    Parameters:
    
        
    Returns:
        
    
    '''

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSwDznLz-GKgWFT1uN0XZYm3bsos899I9MS-pSvEoDC-Cjqo9CWeEuSdjitxjqzF3O39LmjJB0_Fg-B/pub?output=csv')
    
    display(df)
    
    p = input('Which Process would You like to review?')
    v = input('What would you like to return?')
    
    df1 = df[df['Definition']==p]
    
    if v.lower() =='link':
        webbrowser.open(df1['Link'].item())
    elif v.lower() == 'csv':
        return pd.read_csv(df1['CSV'].item())
    elif v.lower()=='streamlit':
        webbrowser.open(df1['Streamlit'].item())

def NavigateUsingDMap():
     
    '''
    Function to Google Mapping Sheet, Navigate to Specific Sites.
    Provides Options, Enable Selection based on inputs.
    
    Parameters:
    
        
    Returns:
        
    
    '''

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSwDznLz-GKgWFT1uN0XZYm3bsos899I9MS-pSvEoDC-Cjqo9CWeEuSdjitxjqzF3O39LmjJB0_Fg-B/pub?output=csv')
    
    display(df)
    
    p = input('Which Process would You like to review?')
    v = input('What would you like to return?')
    
    df1 = df[df['Definition']==p]
    
    if v.lower() =='link':
        webbrowser.open(df1['Link'].item())
    elif v.lower() == 'csv':
        return pd.read_csv(df1['CSV'].item())
    elif v.lower()=='streamlit':
        webbrowser.open(df1['Streamlit'].item())


In [7]:
from data_d_strings import template_doc_string
print(template_doc_string)



    Definition of Function

    Parameters:
        List of Parameters

    Returns:
        Object Type

    date_created:3-Dec-25
    date_last_modified: 3-Dec-25
    classification:TBD
    sub_classification:TBD
    usage:
        Example Function Call



In [22]:
def google_sheet_links(return_value = None):
    
    '''
    Function which Extracts Information Directly From D Google Sheet. If return value is populated and matches one of the Definitions, it will return 
    the File, if not it will return a Dictionary with the Default Arguments. 

    There dictionary was not hard coded as it is subject to continual and preputual change.

    Parameters:
        return_value (str): Value is based on Definition in D Sheet. If value is populated and matched it will return the Google Sheet as a 
        Dataframe, otherwise it will fail and return a dict.

    Returns:
        DataFrame/Dict (Depending on Input)

    date_created:5-Dec-25
    date_last_modified: 5-Dec-25
    classification:TBD
    sub_classification:TBD
    usage:
        d_process_df = google_sheet_links('Processes')

    '''

    # Download CSV from D Google Site
    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSwDznLz-GKgWFT1uN0XZYm3bsos899I9MS-pSvEoDC-Cjqo9CWeEuSdjitxjqzF3O39LmjJB0_Fg-B/pub?output=csv')

    # Return A Dictionary of Google Sheet Objects
    dict_ = df[df['Classification']=='Google Sheet'][['Definition','CSV']].set_index('Definition').to_dict()['CSV']

    if not return_value:
        return dict_

    if return_value:
        try:
            return pd.read_csv(dict_[return_value])
        except:
            print('Could Not Import File from Google')
            return dict_

google_sheet_links('Processes')


,Process,Title,Header,Description
0,BLUE,Linearity,Relationship between Independent and Dependent...,NaN
1,BLUE,Independence.,Observations should be independent of each oth...,NaN
2,BLUE,Homoscedasticity,Variance of the residuals should be constant.,NaN
3,BLUE,Normality of Residuals,Residuals should be normally distributed.,NaN
4,BLUE,No Perfect Collinearity,No perfect linear relationship between residuals,NaN
...,...,...,...,...
66,Solution Adoption,Step 1: Definition,NaN,NaN
67,Solution Adoption,Step 1: Communication,NaN,NaN
68,Solution Adoption,Step 2: Monitoring,NaN,NaN
69,Solution Adoption,Step 3: Analyze Success,NaN,NaN


,Definition,Classification,Description,KEY,Link,CSV,Streamlit
0,Processes,Google Sheet,Defined Processes to Follow. To serve as a fra...,NaN,https://docs.google.com/spreadsheets/d/1iXlm49...,https://docs.google.com/spreadsheets/d/e/2PACX...,https://process-powerpoint-presenetation.strea...
1,Definitions,Google Sheet,"Data Defintions, Links, Images, Etc.",NaN,https://docs.google.com/spreadsheets/d/1tZ-_5V...,https://docs.google.com/spreadsheets/d/e/2PACX...,https://derek-dewald-datadashboard.streamlit.app
2,Notes,Google Sheet,"Notes on a subject, helpful information, thing...",NaN,https://docs.google.com/spreadsheets/d/1jddkkF...,https://docs.google.com/spreadsheets/d/e/2PACX...,NaN
3,Technical Notes,Google Sheet,"Notes Specifically related to Coding, Computer...",NaN,https://docs.google.com/spreadsheets/d/1FpYYq4...,https://docs.google.com/spreadsheets/d/e/2PACX...,https://derekdewald-codedboard.streamlit.app
4,Mapping Sheet,Google Sheet,THIS SHEET.,NaN,https://docs.google.com/spreadsheets/d/1Wfr7Ds...,https://docs.google.com/spreadsheets/d/e/2PACX...,NaN
5,d_py_functions,Folder,Derek's Primary Python Library. \nOriginal Ver...,NaN,NaN,NaN,NaN
6,Job Search,Google Sheet,NaN,NaN,https://docs.google.com/spreadsheets/d/1sMdgmp...,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,https://bconnected.berkeley.edu/services/alumn...,NaN
8,OurWorldData,Data,NaN,NaN,www.ourworlddata.org,NaN,NaN
9,KaggelDatasets,Data,NaN,NaN,https://www.kaggle.com/datasets,NaN,NaN


In [12]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv')
df

,Process,Title,Header,Description
0,Agentic AI,Approaches,Direct User Interaction,NaN
1,Agentic AI,Approaches,Agent/ Assistant Proxy,NaN
2,Agentic AI,Approaches,Agent/ Assistant,NaN
3,Agentic AI,Approaches,Autonomous Agent,NaN
4,Agentic AI,Comparison,LLM,Understanding and Generatic Human Like Text vs...
...,...,...,...,...
304,Type of Problems,Common Types of Problems,Creative,"Innovation, Future Design, where Data might su..."
305,Visualizations,Visualizations,How people Interpret,Christ at Emmaus - Feelings can trump expertis...
306,Visualizations,Visualizations,What Am I looking at.,The bigger display with a wider array of jams ...
307,Visualizations,Visualizations,NaN,Has the performance of the algorithm been asse...


### Definitions

In [ ]:
def GoogleProcessSheetLinks():
     
    '''
    Function to Google Mapping Sheet, Navigate to Specific Sites.
    Provides Options, Enable Selection based on inputs.
    
    Parameters:
    
        
    Returns:
        
    
    '''

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSwDznLz-GKgWFT1uN0XZYm3bsos899I9MS-pSvEoDC-Cjqo9CWeEuSdjitxjqzF3O39LmjJB0_Fg-B/pub?output=csv')
    
    display(df)
    
    p = input('Which Process would You like to review?')
    v = input('What would you like to return?')
    
    df1 = df[df['Definition']==p]
    
    if v.lower() =='link':
        webbrowser.open(df1['Link'].item())
    elif v.lower() == 'csv':
        return pd.read_csv(df1['CSV'].item())
    elif v.lower()=='streamlit':
        webbrowser.open(df1['Streamlit'].item())

#### Notes

In [11]:
Notes = {
    'Topic':'The High Level Topic',
    'Record Type':"Definition of the type of record",
    'Record':'The individual classification to which ',
    'Record Notes':"Detailed Description of related item"
}

In [10]:
df[df['Process']=='Machine Learning']

,Process,Title,Header,Description
91,Machine Learning,Overview,NaN,NaN
92,Machine Learning,Approaches,Supervised Learning,NaN
93,Machine Learning,Approaches,Unsupervised Learning,NaN
94,Machine Learning,Approaches,SemiSupervised Learning,NaN
95,Machine Learning,Approaches,Reinforcement Learning,NaN
96,Machine Learning,Optimization,General,"In 9 dimensions, might not be global minimum, ..."
97,Machine Learning,Optimization,Requirements,Small change hypothesis must hold. Example We...
98,Machine Learning,Optimization,Reminders,"Models will cheat. Self-driving car, multi lan..."
99,Machine Learning,Optimization,Approaches,"L1 Regularization, L2 Regularization"
100,Machine Learning,Optimization,Goal,"What are you attempting to maximize, Training ..."


,Process,Title,Header,Description
0,Agentic AI,Approaches,Direct User Interaction,NaN
1,Agentic AI,Approaches,Agent/ Assistant Proxy,NaN
2,Agentic AI,Approaches,Agent/ Assistant,NaN
3,Agentic AI,Approaches,Autonomous Agent,NaN
4,Agentic AI,Comparison,LLM,Understanding and Generatic Human Like Text vs...
...,...,...,...,...
304,Type of Problems,Common Types of Problems,Creative,"Innovation, Future Design, where Data might su..."
305,Visualizations,Visualizations,How people Interpret,Christ at Emmaus - Feelings can trump expertis...
306,Visualizations,Visualizations,What Am I looking at.,The bigger display with a wider array of jams ...
307,Visualizations,Visualizations,NaN,Has the performance of the algorithm been asse...


In [9]:
df['Title'].value_counts()

Title
Story Telling               35
Evaluation                  17
Tensor                      12
Decision Making             11
Presentation Tenants        10
                            ..
Tweak Model                  1
Validate Model               1
Regression                   1
Dimensionality Reduction     1
Measure Change.              1
Name: count, Length: 89, dtype: int64

In [8]:
df['Process'].value_counts()

Process
TensorFlow                           55
Machine Learning                     40
Communication                        33
Presentation                         24
Tools                                18
Confusion Matrix                     16
Model Types                          13
Kubernetes Production Environment    11
Persuasion                           11
Decision Making                      11
Experimentation                       9
ML Pipeline Cheat Sheet               9
Time Series                           7
ML Tasks                              7
Type of Problems                      6
ML Calculation                        5
Agentic AI                            5
Tensorflow                            4
Sklearn                               4
Visualizations                        4
StoryTelling                          3
Python                                2
Research Design                       2
Math                                  2
ETL                             

In [ ]:
# File Description: 
from IPython.display import display, Markdown, Math, HTML
from Connections import ParamterMapping
import pandas as pd
import re

def D_Notes_Reader(topic=None):
    '''
    Function to read Notes Files Saved in Google Docs

    Parameters: 
        topic (str): Argument to enable Filtering of Returned Dataframe to a Specific Topic. If a Filter is not applied it can be difficult
        to read the output as it's one continuous text without the Header Comments.

    Returns:
        Printed version of Notes (Powerpoint Like Format)

    Date Created: August 17, 2025
    Date Last Modified: 
    
    '''
    
    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv')
    if topic:
        temp = df[df['Process']==topic]
        if len(temp)>0:
            return JupyterNotebookMarkdown(temp)
    else:
        return JupyterNotebookMarkdown(df)
            

def JupyterNotebookMarkdown(df, column_order = ['Title','Header','Description'],return_value=""):
    '''
    Function to Create a Markdown file from Process DF, which is a data frame of the structure, 
    Title, Header, Description

    Args:
        df (DataFrame): Must include columns Title, Header, Description
        return_value (str): 
            If "", renders HTML in notebook.
            If text, returns HTML Markdown string.
    
    Returns:
        str or display: Based on return_value


    Date Created:
    Date Last Maintained: August 17, 2025

    '''
    try:
        df1 = df[column_order]
    except:
        print('DataFrame must include columns: Title, Header, Description')
        return ''

    text = ""
    step_number = 1
    last_title = None
    last_header = None
    open_l2 = False  # Track if L2 <ul> is open
    open_l3 = False  # Track if L3 <ul> is open

    for _, row in df1.iterrows():
        curr_title = row[column_order[0]]
        curr_header = row[column_order[1]]
        curr_description = row[column_order[2]]

        # If new Title
        if curr_title != last_title:
            if open_l3:
                text += "</ul>\n"
                open_l3 = False
            if open_l2:
                text += "</ul>\n"
                open_l2 = False
            if last_title is not None:
                text += "</ul>\n"  # Close previous title's outer <ul>

            text += f"<h4>{step_number}. {curr_title}</h4>\n<ul>\n"
            step_number += 1
            last_title = curr_title
            last_header = None  # Reset header context

        # If new Header
        if curr_header != last_header and isinstance(curr_header, str) and curr_header.strip():
            if open_l3:
                text += "</ul>\n"
                open_l3 = False
            if open_l2:
                text += "</ul>\n"
                open_l2 = False

            text += f"  <ul><li>{curr_header}</li>\n"
            open_l2 = True
            last_header = curr_header

        # If Description exists
        if isinstance(curr_description, str) and curr_description.strip():
            if not open_l3:
                text += "    <ul>\n"
                open_l3 = True
            text += f"      <li>{curr_description}</li>\n"

    # Close any open lists at the end
    if open_l3:
        text += "    </ul>\n"
    if open_l2:
        text += "  </ul>\n"
    text += "</ul>\n"

    if return_value == "":
        display(HTML(text))
    else:
        return text


# File Description: Default Storage Location. Should try my absolute Best NOT to use. But can be a temporary holding spot.


from IPython.display import display, HTML
import pandas as pd
import numpy as np
import datetime

# Will Not work on GitHub
import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions/")

from Connections import ParamterMapping

def JupyterNotebookMarkdown(df,return_value=""):
    
    '''
    Function to Create a Markdown file from Process DF, which is a data frame of the structure, 
    Title,Header,Description
    
    Args:
        Dataframe( Must be of format, Title, Header, Description)
        return_value (str: "" or text):
            If Blank, will render text in HTML Format. 
            If text, then will return text for rendering in HTML Markdown
    
    Returns:
        Conditional on Return Value. Please read Args.
    
    
    '''
    
    try:
        df1 = df[['Title','Header','Description']]
    
    except:
        
        print('DataFrame does not meet structure requirement, which must include 3 Column: Title, Header, Description')
        return ''
    
    title= ""
    step_number = 1
    text = ""

    l2_bullet = '-'  # Level 2 Bullet
    l3_bullet = '*'  # Level 3 Bullet

    for index, row in df1.iterrows():
        # Ensure previous list is closed before starting a new title
        if title and title != row.iloc[0]:  
            text += "</ul>\n"  # Close the last unordered list before switching to a new title

        # If it's a new title, start a new section
        if title == "" or title != row.iloc[0]:
            text += f"<h4>{step_number}. {row.iloc[0]}</h4>\n<ul>\n"  # Reset indentation
            step_number += 1
            title = row.iloc[0]  # Store the new title

        # Add Level 2 content (Column 2)
        if isinstance(row.iloc[1], str) and row.iloc[1].strip():
            text += f"  <li>{row.iloc[1]}</li>\n"  # L2 starts here

            # Add Level 3 content (Column 3) only if it exists
            if isinstance(row.iloc[2], str) and row.iloc[2].strip():
                text += f"    <ul><li>{row.iloc[2]}</li></ul>\n"  # L3 indented under L2

    text += "</ul>\n"  # Close any remaining lists

    if return_value =="":# Display the formatted HTML output in Jupyter Notebook
        display(HTML(text))
        
    else:
        return text
    

def DataFrameFromProcess(process_name=None,
                        return_value = 'Markdown'):
    
    '''
    Function to Extract Data from Process Sheet and Return Markdown Text in Jupyter Notebook.
    Created because Streamlit functionality Changed and could not support HTML Display functionality.
    
    Parameters:
        process_name (str): Value in Column Process, From Google SHeet Process.
        return_value (str): Markdown (returns Markdown value), otherwise it returns filtered dataframe
    
    Returns: 
        Based on return_value and process name. Dataframe, Filtered Data Frame or HTML Markdown text.
    
    '''
    
    try:
        df = pd.read_csv(ParamterMapping('ProcessSheet')['CSV'].item())
    except:
        print('Could Not Extract DataFrame')
    
    if process_name ==None:
        return JupyterNotebookMarkdown(df)
    
    try:
        df1 = df[df['Process']==process_name].copy()
    except:
        print('Could Not Filter Process Name')
        return df
    
    if return_value == 'Markdown':
        try:
            return JupyterNotebookMarkdown(df1)
        except:
            print('Could Not Render JupyterNotebookMarkdown')
            return df1

In [ ]:
import ast
import re
import pandas as pd

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

In [ ]:


import pandas as pd
import numpy as np

def TranposeNonTimeSeriesDF(df, index, columns=None):
    '''
    Transposes a non-time-series DataFrame from wide to long format by melting specified columns.

    This is especially useful for flattening columns into a single column to support tools 
    like Power BI, where long format enables dynamic pivoting and aggregation.

    Parameters:
        df (DataFrame): The input pandas DataFrame.
        index (list): Columns to retain as identifiers (will remain unchanged).
        columns (list): Columns to unpivot into key-value pairs.

    Returns:
        DataFrame: A long-format DataFrame with 'variable' and 'value' columns.
    '''
    if not columns:
        columns = [col for col in final_df1.columns if (isinstance(col, pd.Timestamp))|(isinstance(col, datetime.datetime))]
    
    return df.melt(id_vars=index, value_vars=columns)

def CreatePivotTableFromTimeSeries(df,
                                   index,
                                   columns,
                                   values,
                                   aggfunc='sum',
                                   skipna=True):
    
    '''
    Function to Summaryize a Time Series Dataframe into a Pivot. Creating a number of critical Metrics.
    
    
    
    '''
    
    # 1. Pivot
    if index==None:
        df1 = df.pivot_table(columns=columns,values=values,aggfunc=aggfunc)
    else:
        df1 = df.pivot_table(index=index, columns=columns, values=values, aggfunc=aggfunc)

    # 2. Capture original month columns IMMEDIATELY after pivot
    month_cols = df1.columns.tolist()
 
    # 3. Add rolling window stats
    if len(month_cols) >= 3:
        df1['AVG_3M'] = df1[month_cols[-3:]].mean(axis=1, skipna=skipna)
        df1['CHG_3M'] = df1[month_cols[-1]]-df1[month_cols[-3]]
        try:
            df1['PERC_CHG_3M'] = df1['CHG_3M']/df1[month_cols[-3]]
        except:
            df1['PERC_CHG_3M'] = 0
    
    if len(month_cols) >= 6:
        df1['AVG_6M'] = df1[month_cols[-6:]].mean(axis=1, skipna=skipna)
        df1['CHG_6M'] = df1[month_cols[-1]]-df1[month_cols[-6]]
        try:
            df1['PERC_CHG_6M'] = df1['CHG_6M']/df1[month_cols[-6]]
        except:
            df1['PERC_CHG_6M'] = 0
            
    if len(month_cols) >= 12:
        df1['AVG_12M'] = df1[month_cols[-12:]].mean(axis=1, skipna=skipna)
        df1['CHG_12M'] = df1[month_cols[-1]]-df1[month_cols[-12]]
        try:
            df1['PERC_CHG_12M'] = df1['CHG_12M']/df1[month_cols[-12]]
        except:
            df1['PERC_CHG_12M'] = 0

    df1['CHG_DF']  = df1[month_cols[-1]]-df1[month_cols[0]]
    df1['AVG_DF'] = df1[month_cols[-1:]].mean(axis=1, skipna=skipna)
    df1['PERC_CHG_DF'] = df1['AVG_DF']/df1[month_cols[-1]]

    
    # 4. Now calculate global stats **only using the original month columns**
    stats = pd.DataFrame({
        'MEAN': df1[month_cols].mean(axis=1, skipna=skipna),
        'STD': df1[month_cols].std(axis=1, skipna=skipna),
        'MAX': df1[month_cols].max(axis=1, skipna=skipna),
        'MIN': df1[month_cols].min(axis=1, skipna=skipna),
        'COUNT': df1[month_cols].count(axis=1)
    })

    # 5. Merge the stats
    df1 = pd.concat([df1, stats], axis=1)
    
    return df1.fillna(0)


def CreateMultiplePivotTableFromTimeSeries(df,
                                           index_list,
                                           metric_list,
                                           column):
    '''
    Function to utilize when Attempting to Create Multip[le Times Series. Specifically Multiple Metrics, and Multiple Index's
    



    '''
    

    final_df = pd.DataFrame()
    
    # Iterate through all Possible Metrics Selected.
    
    for metric in metric_list:
        print(f'Attempting to Process:{metric}')
        try:
            all_df = CreatePivotTableFromTimeSeries(df=df,index=None,columns=column,values=metric,aggfunc='sum') 
            cols = list(all_df.columns)
            all_df = all_df.reset_index(drop=True)
            all_df['METRIC'] = metric
            cols.insert(0,'METRIC')

            for key in index_list:
                cols.insert(0,key)
                all_df[key] = 'All'

            final_df = pd.concat([final_df,all_df[cols]])
            # Iterate through all Index Items Individually
            for key in index_list:
                temp = CreatePivotTableFromTimeSeries(df,
                                                      index=key,
                                                      values=metric,
                                                      columns=column).reset_index() 
                for missing in [x for x in index_list if x != key]:
                    temp[missing] = 'All'
                temp['METRIC'] = metric
                final_df = pd.concat([final_df,temp])

            # Add Value for Metric with Entire Index Combination
            temp = CreatePivotTableFromTimeSeries(df,index=index_list,values=metric,columns=column).reset_index()
            temp['METRIC'] = metric
            final_df = pd.concat([final_df,temp])
        except:
            print(f'Could Not Process Metric:{metric}.')

    return final_df




def CreateMultiplePivotTableFromTimeSeries(df,index_list,metric_list,column):
    '''
    Function to utilize when Attempting to Create Multip[le Times Series. Specifically Multiple Metrics, and Multiple Index's
    



    '''
    
    final_df = pd.DataFrame()
    
    # Iterate through all Possible Metrics Selected.
    for metric in metric_list:
        all_df = CreatePivotTableFromTimeSeries(df=df,index=None,columns=column,values=metric,aggfunc='sum') 
        cols = list(all_df.columns)
        all_df = all_df.reset_index(drop=True)
        all_df['METRIC'] = metric
        cols.insert(0,'METRIC')

        for key in index:
            cols.insert(0,key)
            all_df[key] = 'All'

        final_df = pd.concat([final_df,all_df[cols]])

        # Iterate through all Index Items Individually
        for key in index_list:
            temp = CreatePivotTableFromTimeSeries(df,index=key,
                                                  values=metric,
                                                  columns=column).reset_index() 
            for missing in [x for x in index if x != key]:
                temp[missing] = 'All'
            temp['METRIC'] = metric
            final_df = pd.concat([final_df,temp])
        
        # Add Value for Metric with Entire Index Combination
        temp = CreatePivotTableFromTimeSeries(df,index=index_list,values=metric,columns=column).reset_index()
        temp['METRIC'] = metric
        final_df = pd.concat([final_df,temp])
    
    return final_df


def SummarizeTimeSeriesDf(df,
                          summary_cols,
                          primary_key_list):
    '''
    Function to Summarize a Time Series dataframe based on a finite number of identified Columns.
    
    Parameters
        df (Dataframe): TimeSeries in Nature
        summary_cols (List): List of Columns which are to be included in SUmmary
        primary_key_list (list): Primary Key of Dataframe
    
    Returns
        temp_df1: Raw Data of SUmmary Cols with a Count of Observations. If include Month Variable Easy to add to Pivot Table
        summary: Summary (Excluding Primary Key). including Total Observations, MEan, Max, Min.
    
    
    '''
    temp_df = df[summary_cols].copy()
    temp_df['COUNT'] = 1
    
    # Unique Occurances by Pivot Criteria. Important to Include Month
    temp_df1 = temp_df.groupby(summary_cols).sum().reset_index().rename(columns={'COUNT':'TOTAL_DAYS'})
    
    pivot_columns1 = [x for x in summary_cols if x not in primary_key_list]
    
    summary = temp_df1.groupby(pivot_columns1).agg(
        TOTAL=('TOTAL_DAYS', 'count'),
        AVG_DAYS_OPEN=('TOTAL_DAYS', 'mean'),
        MAX_OBS=('TOTAL_DAYS', 'min'),
        MIN_OBS=('TOTAL_DAYS', 'max')).reset_index()
    
    return temp_df1,summary


import pandas as pd
import numpy as np
from scipy.stats import norm

def SampleDataFrame(df, 
                    conf=.95, 
                    me=0.05,
                    mv=0.5,
                    print_=0,
                    new_column_name=""):
    """
    Returns a random sample from a DataFrame based on confidence level and margin of error.

    Parameters:
        df (pd.DataFrame): The dataset to sample from.
        conf(float): Desired Confidence Percentage Level (e.g., 90, 95, 99).
        me (float): Margin of Error, (default is 5%).
        mv (float): Maximum Variability (Expected Level of Default)

    Returns:
        pd.DataFrame: A random sample of the required size.
    """
    df = df.copy()
    
    if not 0 <= mv <= 1:
        raise ValueError("mv (failure rate) must be between 0 and 1.")

    N = len(df)
    if N == 0:
        raise ValueError("DataFrame is empty")

    # Calculate the Z-score based on the confidence level
    z = norm.ppf(1 - (1 - conf) / 2)
    

    # Calculate the initial sample size (without finite population correction)
    n0 = (z**2 * mv * (1 - mv)) / (me**2)
    
    # Apply finite population correction if the population is smaller than 100,000
    if N >= 10000:  # For large populations, skip the correction
        n = int(n0)
    else:
        n = int((n0 * N) / (n0 + N - 1))

    if print_==1:
        print(f"Z-score: {z}")  # Debug Z-score
        print(f"Initial sample size (n0): {n0}")  # Debug n0
        print(f"Sample size with FPC: {n}")  # Debug final sample size
    
    sample = df.sample(n=n, random_state=42)
    
    if len(new_column_name)==0:
        return sample 

    else:
        sample_index = sample.index
        df[new_column_name] = 0
        df.loc[sample_index, new_column_name] = 1
        return df

def ReviewEntireDataframe(df,file_name=None):
    
    final_df = pd.DataFrame()
    
    for column in df.columns:
        start_time = timeit.default_timer()
        temp_df = ColumnStatisticalReview(df,column)
        print(f'Elapsed time to process {column}:{timeit.default_timer() - start_time:,.2f}')
        final_df = pd.concat([final_df,temp_df],axis=1)
    if file_name:
        final_df.to_csv(f"{file_name}.csv")
        
    return final_df

def ColumnStatisticalReview(df,
                            column_name,
                            partitions=10,
                            top_x_records=10,
                            exclude_blanks_from_segments=1,
                            exclude_zeroes_from_segments=1):

    '''
    Function to Conduct a Simple Statistical Review of a Column, Including Understanding the positional distribution
    of values. 

    Args:
        column_name (str): Name of Column

        partitions (int): Number of partitions to include (Decile 10)

        exclude_blanks_from_segments (int): Binary Flag, whether to exclude Blank Values from Segment determination.
        If blank values are excluded it gives a better representation for the members of the set, however it might 
        provide a misleading representation of the population.

        exclude_zeroes_from_segments (int): As above, with respect to 0 values. Is processed after exclude_blanks, as
        such it can include both blanks and true 0 values. 


    '''

    temp_dict = {}
    
    is_numeric = pd.api.types.is_numeric_dtype(df[column_name])
    
    if is_numeric:
        temp_dict['SUM'] = df[column_name].sum()
        temp_dict['MEAN'] = df[column_name].mean()
        temp_dict['STD_DEV'] =  df[column_name].std()
        temp_dict['MEDIAN'] = df[column_name].median()
        temp_dict['MAX'] = df[column_name].max()
        temp_dict['MIN'] = df[column_name].min()
        
    temp_dict['TOTAL_RECORDS'] = len(df)
    temp_dict['UNIQUE_RECORDS'] = len(df.drop_duplicates(column_name))
    temp_dict['NA_RECORDS'] = len(df[df[column_name].isna()])
    temp_dict['NULL_RECORDS'] = len(df[df[column_name].isnull()])
    
    if is_numeric:
        temp_dict['ZERO_RECORDS'] = len(df[df[column_name]==0])
        temp_dict['NON_ZERO_RECORDS'] = len(df[df[column_name]!=0])    

    temp_df = pd.DataFrame(temp_dict.values(),index=temp_dict.keys(),columns=[column_name])
    
    if temp_dict['TOTAL_RECORDS']==len(df[df[column_name].isnull()]):
        return temp_df

    # Add top X records Based on Frequency
    if top_x_records>0:
        top_instances = pd.DataFrame(df[column_name].value_counts(dropna=False).head(top_x_records)).reset_index().rename(columns={column_name:'count','index':column_name})
        if len(top_instances)>0:
            top_instances[column_name] = top_instances.apply(lambda row: f"Value: {row[column_name]}, Frequency: {row['count']}", axis=1)
            top_instances['index'] = [f"Top {x+1}" for x in range(len(top_instances[column_name]))]
            top_instances = top_instances.drop('count',axis=1).set_index('index')
            temp_df = pd.concat([temp_df,top_instances])
        
    if (partitions>0)&(pd.api.types.is_numeric_dtype(df[column_name]))&(temp_dict['UNIQUE_RECORDS']>1):
        segment_df = ColumnPartitioner(df=df,
                                       column_name=column_name,
                                       partitions=partitions,
                                       exclude_blanks=exclude_blanks_from_segments,
                                       exclude_zeros=exclude_zeroes_from_segments,
                                       return_value='')
        
        seg_val_df = ColumnPartitioner(df=df,
                                           column_name=column_name,
                                           partitions=partitions,
                                           exclude_blanks=exclude_blanks_from_segments,
                                           exclude_zeros=exclude_zeroes_from_segments,
                                           return_value='agg_value').rename(columns={'VALUE':column_name})

        return pd.concat([temp_df,segment_df.T,seg_val_df])
    return temp_df

def CompareFunction(func1,func2,additional_records=20):
    
    '''
    Function which Compares 2 Functions and determines if they are different. Specifically, it can help to easily
    Manage Version control of Functions outside of a More robust environment such as GIT.
    

    '''
    
    list1 = FunctionToSTR(func1)
    list2 = FunctionToSTR(func2)
    
    length = max(len(list1),len(list2))
    
    for record in range(0,length):
        if list1[record]==list2[record]:
            if record == (length-1):
                print("All Records Reconcile")
            pass
        else:
            try:
                print(list1[record:record+additional_records])
                print(list2[record:record+additional_records])
            except:
                print(list1[record:record:])
                print(list2[record:record:])
            break


In [ ]:
import pandas as pd
import webbrowser
import datetime
import requests
import os


def DownloadFilesFromGit(user='derek-dewald',
                        repo='Python_Tools',
                        folder='d_py_functions',
                        output_folder=""):
    '''
    Function to Download Files from Github to a dedicated folder. Specifically used when i DO NOT want to formally link to Github.
    
    Parameters:
        User:
        Repo:
        folder:
        output_folder:
        
    Returns:
        Saves files to Output Folder.
    

    '''
    
    if len(output_folder) == 0:
        output_folder = os.getcwd()
    
    api_url = f"https://api.github.com/repos/{user}/{repo}/contents/{folder}"
    response = requests.get(api_url)

    if response.status_code == 200:
        files = response.json()
        py_files = [file for file in files if file['name'].endswith('.py')]

        for file in py_files:
            file_url = file['download_url']
            file_name = file['name']
            file_response = requests.get(file_url)

            if file_response.status_code == 200:
                with open(os.path.join(output_folder, file_name), "w", encoding="utf-8") as f:
                    f.write(file_response.text)
                    
        return True
    else:
        return False

def ParamterMapping(Definition=""):
    
    '''
    Function to Google Mapping Sheet, which is used to store Mappings, Links, etc.
    For both simplicity and Organization
    
    Args:
        Definition (Str): Key word used to Access individual elements
        
    Returns:
        Dataframe, unless Definition is defined, in which case it might be Str.
    
    '''

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSwDznLz-GKgWFT1uN0XZYm3bsos899I9MS-pSvEoDC-Cjqo9CWeEuSdjitxjqzF3O39LmjJB0_Fg-B/pub?output=csv')
    
    # If user has not included a definition, the return entire DF
    if len(Definition)==0:
        return df
    else:
        try:
            df1 = df[df['Definition']==Definition]
            if len(df1)==1:
                if df1['TYPE'].item()=='csv':
                    return pd.read_csv(df1['VALUE'].item())
                else:
                    return df1['VALUE'].item()
        except:
            return df[df['Definition']==Definition] 


def BackUpGoogleSheets(location='/Users/derekdewald/Documents/Python/Github_Repo/CSV Backup Files/'):
    '''
    Function to Create a Backup of Information Stored in Google Sheets.
    
    Parameters:
        None
        
    Returns:
        CSV Files 
    
    '''
    
    df = ParamterMapping()
    
    for row in range(len(df)):
        try:
            file_name = df['Definition'][row]
            file_location = df['CSV'][row]
            month = datetime.datetime.now().strftime('%b-%y')
            
            temp_df = pd.read_csv(file_location)
            temp_df.to_csv(f'{location}{file_name}_{month}.csv',index=False)
            print(f'Back Up Saved, {location}{file_name}')
        except:
            print(f'Counld Not Print Record {row}')





def NavigateUsingDMap():
     
    '''
    Function to Google Mapping Sheet, Navigate to Specific Sites.
    Provides Options, Enable Selection based on inputs.
    
    Parameters:
    
        
    Returns:
        
    
    '''

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSwDznLz-GKgWFT1uN0XZYm3bsos899I9MS-pSvEoDC-Cjqo9CWeEuSdjitxjqzF3O39LmjJB0_Fg-B/pub?output=csv')
    
    display(df)
    
    p = input('Which Process would You like to review?')
    v = input('What would you like to return?')
    
    df1 = df[df['Definition']==p]
    
    if v.lower() =='link':
        webbrowser.open(df1['Link'].item())
    elif v.lower() == 'csv':
        return pd.read_csv(df1['CSV'].item())
    elif v.lower()=='streamlit':
        webbrowser.open(df1['Streamlit'].item())
